In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


Distiller Class

In [2]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

Creating Teacher and Student Model

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen_with_aug = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen_with_aug.flow_from_directory(
    r'/Users/hardikchhipa/Desktop/DR-IMG_SET',
    target_size=(224, 224),
    batch_size=32,
    # color_mode='grayscale',
    class_mode='sparse',
    subset='training'
    ) # set as training data

validation_generator = train_datagen_with_aug.flow_from_directory(
    r'/Users/hardikchhipa/Desktop/DR-IMG_SET', # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    # color_mode='grayscale',
    class_mode='sparse',
    subset='validation'
    ) # set as validation data

#input_img = Input(shape=(224, 224, 3))

Found 2216 images belonging to 5 classes.
Found 554 images belonging to 5 classes.


In [4]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
# from keras.layers.merge import concatenate

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np

In [5]:
# Create the teacher
from tensorflow.keras.models import load_model
#teacher = load_model('/content/drive/MyDrive/RESNET_152(ADAM_0.01).h5')
#teacher = (("/content/drive/MyDrive/RESNET_152(ADAM_0.01).h5"),
   # [
      #  keras.Input(shape=(28, 28, 1)),
       # layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
       # layers.LeakyReLU(alpha=0.2),
      #  layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
       # layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
      #  layers.Flatten(),
       # layers.Dense(10),
   #() ],
    #name="teacher"


from tensorflow.keras.applications import InceptionV3
import tensorflow.keras.applications


# Load DenseNet-169 model without top layers
inceptionv3_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a Sequential model
inceptionv3 = keras.models.Sequential()

# Set layers in densenet169_model to non-trainable
for layer in inceptionv3_model.layers:
    layer.trainable = False

# Add layers to the model
inceptionv3.add(inceptionv3_model)
inceptionv3.add(Flatten())
inceptionv3.add(Dropout(0.2))
inceptionv3.add(Dense=1024, activation='relu')
inceptionv3.add(Dropout(0.2))
inceptionv3.add(Dense(512, activation='relu'))
inceptionv3.add(Dense(5, activation='softmax')))

i.summary()

teacher= xception

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(224, 224, 3)),

        layers.Conv2D(16, (5, 5), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),


        layers.Conv2D(48, (3, 3), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),


        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

        layers.Conv2D(128, (3, 3), strides=(1, 1), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

        layers.Flatten(),
        layers.Dense(600),
        layers.Dense(300),
        layers.Dense(150),
        layers.Dense(5),


    ],
    name="student",
)
student.summary()

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)


83683744/83683744 [==============================] - 21s 0us/step
Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activa

 block4_sepconv1_bn (BatchN  (None, 28, 28, 728)          2912      ['block4_sepconv1[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 block4_sepconv2_act (Activ  (None, 28, 28, 728)          0         ['block4_sepconv1_bn[0][0]']  
 ation)                                                                                           
                                                                                                  
 block4_sepconv2 (Separable  (None, 28, 28, 728)          536536    ['block4_sepconv2_act[0][0]'] 
 Conv2D)                                                                                          
                                                                                                  
 block4_sepconv2_bn (BatchN  (None, 28, 28, 728)          2912      ['block4_sepconv2[0][0]']     
 ormalizat

 block7_sepconv1_act (Activ  (None, 14, 14, 728)          0         ['add_4[0][0]']               
 ation)                                                                                           
                                                                                                  
 block7_sepconv1 (Separable  (None, 14, 14, 728)          536536    ['block7_sepconv1_act[0][0]'] 
 Conv2D)                                                                                          
                                                                                                  
 block7_sepconv1_bn (BatchN  (None, 14, 14, 728)          2912      ['block7_sepconv1[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 block7_sepconv2_act (Activ  (None, 14, 14, 728)          0         ['block7_sepconv1_bn[0][0]']  
 ation)   

                                                                                                  
 block9_sepconv3_bn (BatchN  (None, 14, 14, 728)          2912      ['block9_sepconv3[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 add_7 (Add)                 (None, 14, 14, 728)          0         ['block9_sepconv3_bn[0][0]',  
                                                                     'add_6[0][0]']               
                                                                                                  
 block10_sepconv1_act (Acti  (None, 14, 14, 728)          0         ['add_7[0][0]']               
 vation)                                                                                          
                                                                                                  
 block10_s

 Normalization)                                                                                   
                                                                                                  
 block12_sepconv3_act (Acti  (None, 14, 14, 728)          0         ['block12_sepconv2_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 block12_sepconv3 (Separabl  (None, 14, 14, 728)          536536    ['block12_sepconv3_act[0][0]']
 eConv2D)                                                                                         
                                                                                                  
 block12_sepconv3_bn (Batch  (None, 14, 14, 728)          2912      ['block12_sepconv3[0][0]']    
 Normalization)                                                                                   
          

                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 16)        0         
 D)                                                              
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 re_lu_1 (ReLU)              (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 7, 7, 48)          13872     
                                                                 
 re_lu_2 (ReLU)              (None, 7, 7, 48)          0         
                                                                 
 max_pooli

Train the Teacher Network and Distill the Teacher weights to Student

In [ ]:
 #Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=6,
)

# Distill teacher to student
graph=distiller.fit(train_generator, epochs=25, validation_data = validation_generator)
graph

# Evaluate student on test dataset
#distiller.evaluate(train_generator, epochs=15, validation_data = validation_generator)

Epoch 1/25
70/70 [==============================] - 75s 1s/step - sparse_categorical_accuracy: 0.4598 - student_loss: 1.5231 - distillation_loss: 0.0046 - val_sparse_categorical_accuracy: 0.6245 - val_student_loss: 1.4167
Epoch 2/25
70/70 [==============================] - 77s 1s/step - sparse_categorical_accuracy: 0.5862 - student_loss: 1.4728 - distillation_loss: 0.0073 - val_sparse_categorical_accuracy: 0.5975 - val_student_loss: 1.5624
Epoch 3/25
70/70 [==============================] - 74s 1s/step - sparse_categorical_accuracy: 0.6169 - student_loss: 1.4639 - distillation_loss: 0.0077 - val_sparse_categorical_accuracy: 0.6173 - val_student_loss: 1.4452
Epoch 4/25
70/70 [==============================] - 79s 1s/step - sparse_categorical_accuracy: 0.6155 - student_loss: 1.4521 - distillation_loss: 0.0082 - val_sparse_categorical_accuracy: 0.5722 - val_student_loss: 1.4586
Epoch 5/25
70/70 [==============================] - 87s 1s/step - sparse_categorical_accuracy: 0.6205 - student_

In [3]:
import matplotlib.pyplot as plt

fig1 = plt.gcf()

plt.plot(graph.history['sparse_categorical_accuracy'])
plt.plot(graph.history['val_sparse_categorical_accuracy'])

# plt.axis(ymin=0.4, ymax=1)
plt.grid()

plt.title('OSPS-MicroNet model Accuracy to its teacher ResNet152')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','validation'])

plt.show()

NameError: ignored

<Figure size 640x480 with 0 Axes>

In [4]:
max_acc = max(graph.history['val_sparse_categorical_accuracy'])
print('The highest accuracy achieved for OSPS-MicroNet model Accuracy to its teacher ResNet152 model is',max_acc*100)

NameError: ignored